# Project 4 - Mc907/Mo651 - Mobile Robotics

### Student:
Luiz Eduardo Cartolano - RA: 183012

### Instructor:
Esther Luna Colombini

### Github Link:
[Project Repository](https://github.com/luizcartolano2/mc907-mobile-robotics)

### Youtube Link:
[Link to Video](https://youtu.be/uqNeEhWo0dA)

### Subject of this Work:
The general objective of this work is to implement a deep learning approach for solve the Visual Odometry problem.

### Goals:
1. Implement and evaluate a Deep VO strategy using images from the [AirSim](https://github.com/microsoft/AirSim) simulator.

In [9]:
import pandas as pd
import glob
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.autograd import Function
from torch.autograd import Variable
from torchvision import models
import math

## Data Pre-Processing

### Clean wrong images

While upload images obtained from the AirSim simulator were noted that some of them had failure, so, we have to clean this data to avoid noise in the dataset.

In [47]:
for dt in ['1','2','3','4','5','6']:
    path = 'dataset/'+'seq'+dt+'/'
    print("-------------------------------------------")
    print('|    '+path)
    all_images = glob.glob(path+'images'+'/*')
    df_poses = pd.read_csv(path+'poses.csv')[['ImageFile']].values
    for img in df_poses:
        if not (path+'images/'+img) in all_images:
            print('|        '+img[0])
    print("-------------------------------------------")          

-------------------------------------------
|    dataset/seq1/
|        img__0_1574447220765996000.png
-------------------------------------------
-------------------------------------------
|    dataset/seq2/
-------------------------------------------
-------------------------------------------
|    dataset/seq3/
-------------------------------------------
-------------------------------------------
|    dataset/seq4/
-------------------------------------------
-------------------------------------------
|    dataset/seq5/
-------------------------------------------
-------------------------------------------
|    dataset/seq6/
-------------------------------------------


### Images

In [50]:
def get_image(path,img_size=(1280,384)):
    """
        Function to read an image from a given path.
        
        :param: path - image path
        :param: img_size - image size
        
        :return: img - numpy array with the images pixels (converted to grayscale and normalized)
        
    """
    # read image from path
    img = cv2.imread(path)
    # resize image to a given size
    img = cv2.resize(img, img_size, cv2.INTER_LINEAR)
    # convert image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # normalize image pixels
    img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    return img

In [51]:
def load_images(img_dir, img_size):
    """
        Function to coordinate the load of all the images that are going to be used.
        
        :param: img_dir - path to the directory containing the images
        :param: img_size - image size
        
        :return: images_set - numpy array with all images at the set
        
    """
    print("-----------------------------------")
    print ("|    Loading images from: ", img_dir)
    # create two empty list that are going to be used for save the images
    images= []
    images_set =[]
    # loop to read all the images of the directory
    for img in glob.glob(img_dir+'/*'):
        images.append(get_image(img,img_size))
    # loop on the read images agrupping them two by two
    for i in range(len(images)-1):
        img1 = images[i]
        img2 = images[i+1]
        # concatenate the two images
        img = np.concatenate([img1, img2],axis = -1)
        images_set.append(img)
    print("|    Images count : ",len(images_set))
    print("-----------------------------------")    
    
    # reshape the array of all images
    images_set = np.reshape(images_set, (-1, 6, 384, 1280))


    return images_set

### Pose

In [ ]:
def isRotationMatrix(R):
    """ Checks if a matrix is a valid rotation matrix
        referred from https://www.learnopencv.com/rotation-matrix-to-euler-angles/
    """
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6

In [ ]:
def rotationMatrixToEulerAngles(R):
    """ calculates rotation matrix to euler angles
        referred from https://www.learnopencv.com/rotation-matrix-to-euler-angles
    """
    assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6

    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0

    return np.array([x, y, z])

In [ ]:
def getMatrices(all_poses):
    all_matrices = []
    for i in range(len(all_poses)):
        #print("I: ",i)
        j = all_poses[i]
        #print("J:   ",j)
        p = np.array([j[3], j[7], j[11]])
        #print("P:   ", p)
        R = np.array([[j[0],j[1],j[2]],
                [j[4],j[5],j[6]],
                [j[8],j[9],j[10]]])
        #print("R:   ", R)
        angles = rotationMatrixToEulerAngles(R)
        #print("Angles: ",angles)
        matrix = np.concatenate((p,angles))
        #print("MATRIX: ", matrix)
        all_matrices.append(matrix)
    return all_matrices

In [ ]:
def load_poses(pose_file):
    print ("pose ",pose_file)
    poses = []
    poses_set = []
    with open(pose_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            pose = np.fromstring(line, dtype=float, sep=' ')
            poses.append(pose)
    poses = getMatrices(poses)
    for i in range(len(poses)-1):
        pose1 = poses[i]
        pose2 = poses[i+1]
        finalpose = pose2-pose1
        poses_set.append(finalpose)
    print("poses count: ",len(poses_set))
    return poses_set

### General

In [ ]:
def VODataLoader(datapath,img_size= (1280,384), test=False):
    print (datapath)
    poses_path = os.path.join(datapath,'data_odometry_gray\\dataset\\poses')
    img_path = os.path.join(datapath,'data_odometry_gray\\dataset\\sequences')
    if test:
        sequences = ['03']  #Kindly use this sequence only for testing as this has mininum number of images
    else:
        #Uncomment below and comment the next to next line to work with larger data 
        #sequences= ['01','03','06']
        sequences = ['03']  
        
    images_set = []
    odometry_set = []
    
    for sequence in sequences:
        images_set.append(torch.FloatTensor(load_images(os.path.join(img_path,sequence,'image_0'),img_size)))
        odometry_set.append(torch.FloatTensor(load_poses(os.path.join(poses_path,sequence+'.txt'))))
    
    return images_set, odometry_set